## Setup

In [47]:
from theano.sandbox import cuda
cuda.use('gpu0')

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29



In [48]:
from imp import reload
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

## Setup Data

We are going to use PoC or GTFO, because reasons
https://www.alchemistowl.org/pocorgtfo/

I did have to make a local file so it could be a text document.

Data for the text is pulled locally
/home/jd/aidata/pocgtfo.txt

In [49]:
# Pulls in a file to learn from
path = get_file('pocgtfo.txt', origin="file:///home/jd/aidata/pocgtfo.txt")
text = open(path).read()

# Adds the others. I could make this a loop but im lazy today
path2 = get_file('pocgtfo2.txt', origin="file:///home/jd/aidata/pocgtfo2.txt")
text2 = open(path2).read()

path3 = get_file('pocgtfo3.txt', origin="file:///home/jd/aidata/pocgtfo3.txt")
text3 = open(path3).read()

path4 = get_file('pocgtfo4.txt', origin="file:///home/jd/aidata/pocgtfo4.txt")
text4 = open(path4).read()

path5 = get_file('pocgtfo5.txt', origin="file:///home/jd/aidata/pocgtfo5.txt")
text5 = open(path5).read()

path6 = get_file('pocgtfo6.txt', origin="file:///home/jd/aidata/pocgtfo6.txt")
text6 = open(path6).read()

path7 = get_file('pocgtfo7.txt', origin="file:///home/jd/aidata/pocgtfo7.txt")
text7 = open(path7).read()

path8 = get_file('pocgtfo8.txt', origin="file:///home/jd/aidata/pocgtfo8.txt")
text8 = open(path8).read()

path9 = get_file('pocgtfo9.txt', origin="file:///home/jd/aidata/pocgtfo9.txt")
text9 = open(path9).read()

path10 = get_file('pocgtfo10.txt', origin="file:///home/jd/aidata/pocgtfo10.txt")
text10 = open(path10).read()

text = text+text2+text3+text4+text5+text6+text7+text8+text9+text10
print('corpus length:', len(text))



corpus length: 1067271


In [50]:
# gets the total number of characters fond in the document
# There are many unique chars because of the Russian and coding.
chars = sorted(list(set(text)))
vocab_size = len(chars)+1
print('total chars:', vocab_size)

total chars: 202


In [51]:
#Places in the 0 chars for padding
chars.insert(0, "\0")

In [52]:
# Here is the list of chars
# Check out how weird they are
''.join(chars[1:-6])

'\n !"#$%&\'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\\]^_`abcdefghijklmnopqrstuvwxyz{|}~\x80\x81\x82¢£¨«´µ·»ÆÈÉ×àáãéïóõıőˆ˙˜αβγζθλπωϕАБВГДЕЗИЙКЛМНОПРСТУФХЦЧШЫЭЯавдезиклмнорстшё–—‘’“”•→∀∆∈−∗√≡≤⊕␣\uf731\uf734'

In [53]:
# creates the chars to indicies and back. Lets us make arrays of numbers not chars
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [54]:
# the idx holds the text
idx = [char_indices[c] for c in text]

In [55]:
idx[:10]

[43, 80, 86, 71, 84, 80, 67, 86, 75, 81]

In [56]:
# If everything is working correctly we will have the first sentance of the document
''.join(indices_char[i] for i in idx[:200])

'International Journal of PoC k GTFO\nIssue 0x00, a CFP with PoC\nAn epistle from the desk of Rt. Revd. Pastor Manul Laphroaig\npastor@phrack.org\nAugust 5, 2013\nLegal Note: Permission to use all or part o'

The data appears to have been properly loaded.

#### 4 Char Model

Lets make inputs; However we are doing 4 not 3

In [57]:
# How many chars are we doing
cs=4
#For the first char... pull back the length -1. Starts chopping up the sentances
c1_dat = [idx[i] for i in range(0, len(idx)-1-cs, cs)]
c2_dat = [idx[i+1] for i in range(0, len(idx)-1-cs, cs)]
c3_dat = [idx[i+2] for i in range(0, len(idx)-1-cs, cs)]
c4_dat = [idx[i+3] for i in range(0, len(idx)-1-cs, cs)]
c5_dat = [idx[i+3] for i in range(0, len(idx)-1-cs, cs)]

In [58]:
# Our Input 
x1 = np.stack(c1_dat[:-2])
x2 = np.stack(c2_dat[:-2])
x3 = np.stack(c3_dat[:-2])
x4 = np.stack(c4_dat[:-2])

In [59]:
# our output
y = np.stack(c5_dat[:-2])

In [60]:
x1[:4], x2[:4], x3[:4], x4[:4]

(array([43, 84, 75, 78]),
 array([80, 80, 81,  2]),
 array([86, 67, 80, 44]),
 array([71, 86, 67, 81]))

In [61]:
y[:4]

array([71, 86, 67, 81])

In [62]:
x1.shape, y.shape

((266815,), (266815,))

In [63]:
# Latent Factors
n_fac = 42

In [64]:
def embedding_input(name, n_in, n_out):
    inp = Input(shape=(1,), dtype='int64', name=name)
    emb = Embedding(n_in, n_out, input_length=1)(inp)
    return inp, Flatten()(emb)

In [65]:
c1_in, c1 = embedding_input('c1', vocab_size, n_fac)
c2_in, c2 = embedding_input('c2', vocab_size, n_fac)
c3_in, c3 = embedding_input('c3', vocab_size, n_fac)
c4_in, c4 = embedding_input('c4', vocab_size, n_fac)

## Create and train model
Pick a size for hidden state

In [66]:
n_hidden = 256

In [67]:
#Creates the first arrow in from the model
dense_in = Dense(n_hidden, activation='relu')

In [68]:
# Create the hidden activation
c1_hidden = dense_in(c1)

In [69]:
# Orange arrows
dense_hidden = Dense(n_hidden, activation='tanh')

The other hidden activations

In [70]:
c2_dense = dense_in(c2)
hidden_2 = dense_hidden(c1_hidden)
c2_hidden = merge([c2_dense, hidden_2])

In [71]:
c3_dense = dense_in(c3)
hidden_3 = dense_hidden(c2_hidden)
c3_hidden = merge([c3_dense, hidden_3])

In [72]:
c4_dense = dense_in(c4)
hidden_4 = dense_hidden(c3_hidden)
c4_hidden = merge([c4_dense, hidden_4])

The last arrow of the diagram

In [73]:
dense_out = Dense(vocab_size, activation='softmax')

In [74]:
# Put in the last layer that goes out
c5_out = dense_out(c4_hidden)

In [75]:
# The model will have 4 inputs and 1 output
model = Model([c1_in, c2_in, c3_in, c4_in], c5_out)

# 4 Input

In [76]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [ ]:
# Let a proper learning rate for the model
model.optimizer.lr=0.000001

In [ ]:
# THis runs very fast so I will run lots of them
model.fit([x1, x2, x3, x4], y, batch_size=64, nb_epoch=8)

In [ ]:
#Put in a steeper learning curve
model.optimizer.lr=0.01

In [ ]:
#Run the model a few times
model.fit([x1, x2, x3, x4], y, batch_size=64, nb_epoch=8)

In [ ]:
# Get a detailed optimizer
model.optimizer.lr=(0.000001)

In [ ]:
model.fit([x1, x2, x3, x4], y, batch_size=64, nb_epoch=8)

In [ ]:
model.optimizer.lr=(0.01)

In [ ]:
model.fit([x1, x2, x3, x4], y, batch_size=64, nb_epoch=8)

###  Test the Model

In [ ]:
# a function to test the next one
def get_next(inp):
    idxs = [char_indices[c] for c in inp]
    arrs = [np.array(i)[np.newaxis] for i in idxs]
    p = model.predict(arrs)
    i = np.argmax(p)
    return chars[i]

In [ ]:
# book
get_next(' boo')

In [ ]:
current = ''
check ='k'
chance = 0
while (current != check):
    model.fit([x1, x2, x3, x4], y, batch_size=64, nb_epoch=1,verbose=0)
    current = get_next(' boo')
    chance = chance +1
    print ("Run "+str(chance) +": Current letter is "+current)

In [ ]:
#Worship
get_next('orsh')

In [ ]:
#Goodspeed
get_next('Good')

In [ ]:
#**ck Out
get_next('uk O')

In [ ]:
#**ck Out
get_next('k Ou')

In [ ]:
# **** Out ... something should go after
get_next(' Out')

In [ ]:
#Pastor Manul Laphroaig
get_next('Past')

## back to three!
Something is very weird with 4. Lets try 3

In [78]:
## Create and train model
#Pick a size for hidden state

n_hidden = 256
cs=3

#Creates the first arrow in from the model
dense_in = Dense(n_hidden, activation='relu')

# Create the hidden activation
c1_hidden = dense_in(c1)

# Orange arrows
dense_hidden = Dense(n_hidden, activation='tanh')

#The other activations

c2_dense = dense_in(c2)
hidden_2 = dense_hidden(c1_hidden)
c2_hidden = merge([c2_dense, hidden_2])

c3_dense = dense_in(c3)
hidden_3 = dense_hidden(c2_hidden)
c3_hidden = merge([c3_dense, hidden_3])

dense_out = Dense(vocab_size, activation='softmax')

# Put in the last layer that goes out
c4_out = dense_out(c3_hidden)

# The model will have 4 inputs and 1 output
model = Model([c1_in, c2_in, c3_in], c4_out)

model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

# Let a proper learning rate for the model
model.optimizer.lr=0.000001

In [79]:
model.optimizer.lr=(0.01)
model.fit([x1, x2, x3], y, batch_size=64, nb_epoch=8)

Epoch 1/8
266815/266815 [==============================] - 8s - loss: 2.8032     
Epoch 2/8
266815/266815 [==============================] - 8s - loss: 2.8597     
Epoch 3/8
266815/266815 [==============================] - 8s - loss: 3.2028     
Epoch 4/8
266815/266815 [==============================] - 8s - loss: 3.2805     
Epoch 5/8
266815/266815 [==============================] - 8s - loss: 3.3346     
Epoch 6/8
266815/266815 [==============================] - 8s - loss: 3.2611     
Epoch 7/8
266815/266815 [==============================] - 8s - loss: 3.2555     
Epoch 8/8
266815/266815 [==============================] - 8s - loss: 3.2816     


In [80]:
model.optimizer.lr=(0.00001)
model.fit([x1, x2, x3], y, batch_size=64, nb_epoch=1)

Epoch 1/1
266815/266815 [==============================] - 8s - loss: 3.2745     


In [81]:
model.optimizer.lr=(0.01)
model.fit([x1, x2, x3], y, batch_size=64, nb_epoch=1)

Epoch 1/1
266815/266815 [==============================] - 8s - loss: 3.2775     


In [82]:
model.optimizer.lr=(0.000001)
model.fit([x1, x2, x3], y, batch_size=64, nb_epoch=2)

Epoch 1/2
266815/266815 [==============================] - 8s - loss: 3.2781     
Epoch 2/2
266815/266815 [==============================] - 8s - loss: 3.2798     


### Try again

In [83]:
# a function to test the next one
def get_next(inp):
    idxs = [char_indices[c] for c in inp]
    arrs = [np.array(i)[np.newaxis] for i in idxs]
    p = model.predict(arrs)
    i = np.argmax(p)
    return chars[i]

In [84]:
#Worship
get_next('rsh')

'e'

In [38]:
current = ''
check ='k'
chance = 0
while (current != check):
    model.fit([x1, x2, x3], y, batch_size=64, nb_epoch=1,verbose=0)
    current = get_next('boo')
    chance = chance +1
    print ("Run "+str(chance) +": Current letter is "+current)

Run 1: Current letter is d
Run 2: Current letter is r
Run 3: Current letter is  
Run 4: Current letter is n
Run 5: Current letter is  
Run 6: Current letter is t
Run 7: Current letter is n
Run 8: Current letter is k


In [85]:
#Goodspeed
get_next('Goo')

'r'

In [86]:
model.fit([x1, x2, x3], y, batch_size=64, nb_epoch=1)

Epoch 1/1
266815/266815 [==============================] - 8s - loss: 3.2756     


# Time for our RNNS

## Create Inputs

Size of the unrolled RNN

In [87]:
# Number of letters used in the RNN
cs=8

In [91]:
#For each of 0 through 7, create a list of every 8th character with that starting point. 
# These will be the 8 inputs to out model.
c_in_dat = [[idx[i+n] for i in range(0, len(idx)-1-cs, cs)]
            for n in range(cs)]

In [92]:
# writes the outputs for the model
c_out_dat = [idx[i+cs] for i in range(0, len(idx)-1-cs, cs)]

In [96]:
#gets the stack 
xs = [np.stack(c[:-2]) for c in c_in_dat]

In [97]:
# Breaks it into sets of 8
len(xs), xs[0].shape

(8, (133406,))

In [98]:
y = np.stack(c_out_dat[:-2])

In [99]:
# Sets up lists of characters for input
[xs[n][:cs] for n in range(cs)]

[array([43, 75, 87,  2, 54, 71, 67, 86]),
 array([80, 81, 84, 50, 40,  2,  2, 74]),
 array([86, 80, 80, 81, 49, 18, 37,  2]),
 array([71, 67, 67, 37,  1, 90, 40, 50]),
 array([84, 78, 78,  2, 43, 18, 50, 81]),
 array([80,  2,  2, 77, 85, 18,  2, 37]),
 array([67, 44, 81,  2, 85, 14, 89,  1]),
 array([86, 81, 72, 41, 87,  2, 75, 35])]

In [100]:
# output
y[:cs]

array([75, 87,  2, 54, 71, 67, 86, 80])

In [101]:
n_fac = 42

### Create and train model

In [102]:
def embedding_input(name, n_in, n_out):
    inp = Input(shape=(1,), dtype='int64', name=name+'_in')
    emb = Embedding(n_in, n_out, input_length=1, name=name+'_emb')(inp)
    return inp, Flatten()(emb)

In [103]:
c_ins = [embedding_input('c'+str(n), vocab_size, n_fac) for n in range(cs)]

In [104]:
n_hidden = 256

In [105]:
# Setting up the RNN model.
# Dense in is the the first layer that shoots in
# Dense hidden is circling around inside
# Dense out it the exit
dense_in = Dense(n_hidden, activation='relu')
dense_hidden = Dense(n_hidden, activation='relu', init='identity')
dense_out = Dense(vocab_size, activation='softmax')

In [106]:
# Starts our the first input as all 0s
hidden = dense_in(c_ins[0][1])

In [107]:
# Remember CS is the number of iterations
# This runs the loops
for i in range(1,cs):
    c_dense = dense_in(c_ins[i][1])
    hidden = dense_hidden(hidden)
    hidden = merge([c_dense, hidden])

In [108]:
# final stage to push it out
c_out = dense_out(hidden)

### Model Generation

In [109]:
model = Model([c[0] for c in c_ins], c_out)
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [110]:
model.fit(xs, y, batch_size=64, nb_epoch=12)

Epoch 1/12
133406/133406 [==============================] - 8s - loss: 2.7434     
Epoch 2/12
133406/133406 [==============================] - 8s - loss: 2.4859     
Epoch 3/12
133406/133406 [==============================] - 8s - loss: 2.3816     
Epoch 4/12
133406/133406 [==============================] - 8s - loss: 2.3075     
Epoch 5/12
133406/133406 [==============================] - 7s - loss: 2.2476     
Epoch 6/12
133406/133406 [==============================] - 7s - loss: 2.1991     
Epoch 7/12
133406/133406 [==============================] - 7s - loss: 2.1567     
Epoch 8/12
133406/133406 [==============================] - 7s - loss: 2.1215     
Epoch 9/12
133406/133406 [==============================] - 7s - loss: 2.0900     
Epoch 10/12
133406/133406 [==============================] - 7s - loss: 2.0635     
Epoch 11/12
133406/133406 [==============================] - 8s - loss: 2.0390     
Epoch 12/12
133406/133406 [==============================] - 7s - loss: 2.0172     


### Test it

In [111]:
def get_next(inp):
    idxs = [np.array(char_indices[c])[np.newaxis] for c in inp]
    p = model.predict(idxs)
    return chars[np.argmax(p)]

In [112]:
get_next('Internat')

'i'

In [113]:
get_next('nternati')

'n'

In [114]:
get_next('Pastor L')

'a'

# RNN With Keras

In [115]:
# The Vocab size is dependent on the file.
n_hidden, n_fac, cs, vocab_size = (256, 42, 8, 203)

In [118]:
# Sequential model 
model=Sequential([
        Embedding(vocab_size, n_fac, input_length=cs),
        SimpleRNN(n_hidden, activation='relu', inner_init='identity'),
        Dense(vocab_size, activation='softmax')
    ])

In [119]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_10 (Embedding)         (None, 8, 42)         8526        embedding_input_2[0][0]          
____________________________________________________________________________________________________
simplernn_2 (SimpleRNN)          (None, 256)           76544       embedding_10[0][0]               
____________________________________________________________________________________________________
dense_20 (Dense)                 (None, 203)           52171       simplernn_2[0][0]                
Total params: 137,241
Trainable params: 137,241
Non-trainable params: 0
____________________________________________________________________________________________________


In [120]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [121]:
model.fit(np.concatenate(xs,axis=1), y, batch_size=64, nb_epoch=8)

Epoch 1/8
133406/133406 [==============================] - 10s - loss: 2.9453    
Epoch 2/8
133406/133406 [==============================] - 10s - loss: 2.4588    
Epoch 3/8
133406/133406 [==============================] - 10s - loss: 2.2626    
Epoch 4/8
133406/133406 [==============================] - 10s - loss: 2.1263    
Epoch 5/8
133406/133406 [==============================] - 10s - loss: 2.0258    
Epoch 6/8
133406/133406 [==============================] - 10s - loss: 1.9463    
Epoch 7/8
133406/133406 [==============================] - 10s - loss: 1.8833    
Epoch 8/8
133406/133406 [==============================] - 10s - loss: 1.8314    


In [122]:
def get_nexts_keras(inp):
    idxs = [char_indices[c] for c in inp]
    arr = np.array(idxs)[np.newaxis,:]
    p = model.predict(arr)[0]
    print(list(inp))
    return [chars[np.argmax(o)] for o in p]

In [126]:
# Seems off
get_nexts_keras('this is ')

['t', 'h', 'i', 's', ' ', 'i', 's', ' ']


['\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 '\x00',
 

# Return Sequences

## Create Inputs

In [127]:
c_out_dat = [[idx[i+n] for i in range(1, len(idx)-cs, cs)]
            for n in range(cs)]

In [128]:
ys = [np.stack(c[:-2]) for c in c_out_dat]

In [129]:
[xs[n][:cs] for n in range(cs)]

[array([[43],
        [75],
        [87],
        [ 2],
        [54],
        [71],
        [67],
        [86]]), array([[80],
        [81],
        [84],
        [50],
        [40],
        [ 2],
        [ 2],
        [74]]), array([[86],
        [80],
        [80],
        [81],
        [49],
        [18],
        [37],
        [ 2]]), array([[71],
        [67],
        [67],
        [37],
        [ 1],
        [90],
        [40],
        [50]]), array([[84],
        [78],
        [78],
        [ 2],
        [43],
        [18],
        [50],
        [81]]), array([[80],
        [ 2],
        [ 2],
        [77],
        [85],
        [18],
        [ 2],
        [37]]), array([[67],
        [44],
        [81],
        [ 2],
        [85],
        [14],
        [89],
        [ 1]]), array([[86],
        [81],
        [72],
        [41],
        [87],
        [ 2],
        [75],
        [35]])]

In [130]:
[ys[n][:cs] for n in range(cs)]

[array([80, 81, 84, 50, 40,  2,  2, 74]),
 array([86, 80, 80, 81, 49, 18, 37,  2]),
 array([71, 67, 67, 37,  1, 90, 40, 50]),
 array([84, 78, 78,  2, 43, 18, 50, 81]),
 array([80,  2,  2, 77, 85, 18,  2, 37]),
 array([67, 44, 81,  2, 85, 14, 89,  1]),
 array([86, 81, 72, 41, 87,  2, 75, 35]),
 array([75, 87,  2, 54, 71, 67, 86, 80])]

### Create and train model

In [131]:
dense_in = Dense(n_hidden, activation='relu')
dense_hidden = Dense(n_hidden, activation='relu', init='identity')
dense_out = Dense(vocab_size, activation='softmax', name='output')

In [132]:
inp1 = Input(shape=(n_fac,), name='zeros')
hidden = dense_in(inp1)

In [133]:
outs = []

for i in range(cs):
    c_dense = dense_in(c_ins[i][1])
    hidden = dense_hidden(hidden)
    hidden = merge([c_dense, hidden], mode='sum')
    # every layer now has an output
    outs.append(dense_out(hidden))

In [134]:
model = Model([inp1] + [c[0] for c in c_ins], outs)
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [135]:
zeros = np.tile(np.zeros(n_fac), (len(xs[0]),1))
zeros.shape

(133406, 42)

In [136]:
model.fit([zeros]+xs, ys, batch_size=64, nb_epoch=12)

Epoch 1/12
133406/133406 [==============================] - 20s - loss: 21.7578 - output_loss_1: 2.9341 - output_loss_2: 2.7872 - output_loss_3: 2.7268 - output_loss_4: 2.6858 - output_loss_5: 2.6620 - output_loss_6: 2.6616 - output_loss_7: 2.6529 - output_loss_8: 2.6473    
Epoch 2/12
133406/133406 [==============================] - 20s - loss: 19.8388 - output_loss_1: 2.7743 - output_loss_2: 2.6178 - output_loss_3: 2.5049 - output_loss_4: 2.4272 - output_loss_5: 2.3913 - output_loss_6: 2.3823 - output_loss_7: 2.3718 - output_loss_8: 2.3691    
Epoch 3/12
133406/133406 [==============================] - 20s - loss: 19.2321 - output_loss_1: 2.7590 - output_loss_2: 2.5914 - output_loss_3: 2.4473 - output_loss_4: 2.3405 - output_loss_5: 2.2899 - output_loss_6: 2.2754 - output_loss_7: 2.2647 - output_loss_8: 2.2638    
Epoch 4/12
133406/133406 [==============================] - 20s - loss: 18.8639 - output_loss_1: 2.7511 - output_loss_2: 2.5797 - output_loss_3: 2.4168 - output_loss_4: 2.2

### Test model

In [137]:
def get_nexts(inp):
    idxs = [char_indices[c] for c in inp]
    arrs = [np.array(i)[np.newaxis] for i in idxs]
    p = model.predict([np.zeros(n_fac)[np.newaxis,:]] + arrs)
    print(list(inp))
    return [chars[np.argmax(o)] for o in p]

In [138]:
get_nexts(' this is')

[' ', 't', 'h', 'i', 's', ' ', 'i', 's']


['t', 'h', 'e', 't', ' ', 'i', 's', ' ']

In [139]:
get_nexts(' part of')

[' ', 'p', 'a', 'r', 't', ' ', 'o', 'f']


['t', 'e', 'r', 'e', 'i', 'o', 'f', ' ']

### Sequence Model with Keras

In [140]:
n_hidden, n_fac, cs, vocab_size

(256, 42, 8, 203)

In [141]:
model=Sequential([
        Embedding(vocab_size, n_fac, input_length=cs),
        SimpleRNN(n_hidden, return_sequences=True, activation='relu', inner_init='identity'),
        TimeDistributed(Dense(vocab_size, activation='softmax')),
    ])

In [142]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_11 (Embedding)         (None, 8, 42)         8526        embedding_input_3[0][0]          
____________________________________________________________________________________________________
simplernn_3 (SimpleRNN)          (None, 8, 256)        76544       embedding_11[0][0]               
____________________________________________________________________________________________________
timedistributed_1 (TimeDistribut (None, 8, 203)        52171       simplernn_3[0][0]                
Total params: 137,241
Trainable params: 137,241
Non-trainable params: 0
____________________________________________________________________________________________________


In [143]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [144]:
xs[0].shape

(133406, 1)

In [145]:
## stack is changed to concatenate
x_rnn=np.concatenate(xs, axis=1)
y_rnn=np.expand_dims(np.concatenate(ys, axis=1), -1)

In [146]:
x_rnn.shape, y_rnn.shape

((133406, 8), (133406, 8, 1))

In [147]:
model.fit(x_rnn, y_rnn, batch_size=64, nb_epoch=10)

Epoch 1/10
133406/133406 [==============================] - 12s - loss: 2.6522    
Epoch 2/10
133406/133406 [==============================] - 12s - loss: 2.2407    
Epoch 3/10
133406/133406 [==============================] - 12s - loss: 2.1205    
Epoch 4/10
133406/133406 [==============================] - 12s - loss: 2.0599    
Epoch 5/10
133406/133406 [==============================] - 12s - loss: 2.0227    
Epoch 6/10
133406/133406 [==============================] - 12s - loss: 1.9968    
Epoch 7/10
133406/133406 [==============================] - 12s - loss: 1.9769    
Epoch 8/10
133406/133406 [==============================] - 12s - loss: 1.9622    
Epoch 9/10
133406/133406 [==============================] - 12s - loss: 1.9494    
Epoch 10/10
133406/133406 [==============================] - 12s - loss: 1.9390    


In [148]:
def get_nexts_keras(inp):
    idxs = [char_indices[c] for c in inp]
    arr = np.array(idxs)[np.newaxis,:]
    p = model.predict(arr)[0]
    print(list(inp))
    return [chars[np.argmax(o)] for o in p]

In [149]:
get_nexts_keras(' this is')

[' ', 't', 'h', 'i', 's', ' ', 'i', 's']


['t', 'h', 'e', 's', ' ', 'i', 's', ' ']

### One Hot sequence model with keras

In [150]:
model=Sequential([
        SimpleRNN(n_hidden, return_sequences=True, input_shape=(cs, vocab_size),
                  activation='relu', inner_init='identity'),
        TimeDistributed(Dense(vocab_size, activation='softmax')),
    ])
model.compile(loss='categorical_crossentropy', optimizer=Adam())

In [151]:
oh_ys = [to_categorical(o, vocab_size) for o in ys]
oh_y_rnn=np.stack(oh_ys, axis=1)

oh_xs = [to_categorical(o, vocab_size) for o in xs]
oh_x_rnn=np.stack(oh_xs, axis=1)

oh_x_rnn.shape, oh_y_rnn.shape

((133406, 8, 203), (133406, 8, 203))

In [155]:
model.fit(oh_x_rnn, oh_y_rnn, batch_size=64, nb_epoch=8)

Epoch 1/8
133406/133406 [==============================] - 11s - loss: 1.9549    
Epoch 2/8
133406/133406 [==============================] - 12s - loss: 1.9429    
Epoch 3/8
133406/133406 [==============================] - 12s - loss: 1.9325    
Epoch 4/8
133406/133406 [==============================] - 12s - loss: 1.9238    
Epoch 5/8
133406/133406 [==============================] - 12s - loss: 1.9160    
Epoch 6/8
133406/133406 [==============================] - 12s - loss: 1.9093    
Epoch 7/8
133406/133406 [==============================] - 12s - loss: 1.9033    
Epoch 8/8
133406/133406 [==============================] - 12s - loss: 1.8978    


In [156]:
def get_nexts_oh(inp):
    idxs = np.array([char_indices[c] for c in inp])
    arr = to_categorical(idxs, vocab_size)
    p = model.predict(arr[np.newaxis,:])[0]
    print(list(inp))
    return [chars[np.argmax(o)] for o in p]

In [157]:
get_nexts_oh(' this is')

[' ', 't', 'h', 'i', 's', ' ', 'i', 's']


['t', 'h', 'e', 's', ' ', 'i', 's', ' ']

## Stateful model with keras


In [158]:
bs=64

In [159]:
model=Sequential([
        Embedding(vocab_size, n_fac, input_length=cs, batch_input_shape=(bs,8)),
        BatchNormalization(),
        LSTM(n_hidden, return_sequences=True, stateful=True),
        TimeDistributed(Dense(vocab_size, activation='softmax')),
    ])

In [160]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [161]:
mx = len(x_rnn)//bs*bs

In [163]:
model.fit(x_rnn[:mx], y_rnn[:mx], batch_size=bs, nb_epoch=4, shuffle=False)

Epoch 1/4
133376/133376 [==============================] - 35s - loss: nan    
Epoch 2/4
133376/133376 [==============================] - 35s - loss: nan    
Epoch 3/4
133376/133376 [==============================] - 34s - loss: nan    
Epoch 4/4
133376/133376 [==============================] - 34s - loss: nan    


In [164]:
model.optimizer.lr=1e-4

In [165]:
model.fit(x_rnn[:mx], y_rnn[:mx], batch_size=bs, nb_epoch=1, shuffle=False)

Epoch 1/1
133376/133376 [==============================] - 34s - loss: nan    


In [166]:
model.fit(x_rnn[:mx], y_rnn[:mx], batch_size=bs, nb_epoch=1, shuffle=False)

Epoch 1/1
133376/133376 [==============================] - 34s - loss: nan    


## Theano RNN

In [167]:
n_input = vocab_size
n_output = vocab_size

In [168]:
def init_wgts(rows, cols): 
    scale = math.sqrt(2/rows)
    return shared(normal(scale=scale, size=(rows, cols)).astype(np.float32))
def init_bias(rows): 
    return shared(np.zeros(rows, dtype=np.float32))

In [169]:
def wgts_and_bias(n_in, n_out): 
    return init_wgts(n_in, n_out), init_bias(n_out)
def id_and_bias(n): 
    return shared(np.eye(n, dtype=np.float32)), init_bias(n)

In [170]:
t_inp = T.matrix('inp')
t_outp = T.matrix('outp')
t_h0 = T.vector('h0')
lr = T.scalar('lr')

all_args = [t_h0, t_inp, t_outp, lr]

In [171]:
W_h = id_and_bias(n_hidden)
W_x = wgts_and_bias(n_input, n_hidden)
W_y = wgts_and_bias(n_hidden, n_output)
w_all = list(chain.from_iterable([W_h, W_x, W_y]))

In [172]:
def step(x, h, W_h, b_h, W_x, b_x, W_y, b_y):
    # Calculate the hidden activations
    h = nnet.relu(T.dot(x, W_x) + b_x + T.dot(h, W_h) + b_h)
    # Calculate the output activations
    y = nnet.softmax(T.dot(h, W_y) + b_y)
    # Return both (the 'Flatten()' is to work around a theano bug)
    return h, T.flatten(y, 1)

In [173]:
[v_h, v_y], _ = theano.scan(step, sequences=t_inp, 
                            outputs_info=[t_h0, None], non_sequences=w_all)

In [174]:
error = nnet.categorical_crossentropy(v_y, t_outp).sum()
g_all = T.grad(error, w_all)

In [175]:
def upd_dict(wgts, grads, lr): 
    return OrderedDict({w: w-g*lr for (w,g) in zip(wgts,grads)})

upd = upd_dict(w_all, g_all, lr)

In [176]:
fn = theano.function(all_args, error, updates=upd, allow_input_downcast=True)

In [177]:
X = oh_x_rnn
Y = oh_y_rnn
X.shape, Y.shape

((133406, 8, 203), (133406, 8, 203))

In [178]:
err=0.0; l_rate=0.01
for i in range(len(X)): 
    err+=fn(np.zeros(n_hidden), X[i], Y[i], l_rate)
    if i % 1000 == 999: 
        print ("Error:{:.3f}".format(err/1000))
        err=0.0

Error:27.399
Error:23.952
Error:21.156
Error:22.664
Error:21.519
Error:21.515
Error:20.836
Error:20.436
Error:21.475
Error:20.412
Error:20.212
Error:19.883
Error:19.962
Error:19.435
Error:22.186
Error:20.226
Error:19.549
Error:20.985
Error:18.726
Error:17.993
Error:19.166
Error:17.502
Error:21.074
Error:19.960
Error:16.165
Error:17.212
Error:17.942
Error:19.315
Error:19.140
Error:18.905
Error:18.527
Error:17.819
Error:17.489
Error:16.178
Error:17.781
Error:16.989
Error:17.986
Error:17.998
Error:17.987
Error:17.458
Error:17.858
Error:17.657
Error:17.915
Error:18.928
Error:18.908
Error:18.993
Error:16.183
Error:17.162
Error:18.397
Error:21.384
Error:18.144
Error:16.875
Error:19.213
Error:17.336
Error:17.755
Error:17.029
Error:17.203
Error:16.558
Error:17.488
Error:17.482
Error:17.367
Error:17.092
Error:17.073
Error:18.057
Error:16.982
Error:16.176
Error:16.810
Error:17.777
Error:18.237
Error:17.600
Error:16.836
Error:17.168
Error:19.327
Error:18.336
Error:17.958
Error:17.005
Error:15.579

KeyboardInterrupt: 

In [179]:
f_y = theano.function([t_h0, t_inp], v_y, allow_input_downcast=True)

In [180]:
pred = np.argmax(f_y(np.zeros(n_hidden), X[6]), axis=1)

In [181]:
act = np.argmax(X[6], axis=1)

In [184]:
[indices_char[o] for o in act]

['a', ' ', 'C', 'F', 'P', ' ', 'w', 'i']

In [185]:
[indices_char[o] for o in pred]

['n', 's', ' ', ' ', ' ', 'a', 'o', 'l']

## Pure python RNN!

In [186]:
# Just in Numpy
def sigmoid(x): return 1/(1+np.exp(-x))
def sigmoid_d(x): 
    output = sigmoid(x)
    return output*(1-output)

In [187]:
def relu(x): return np.maximum(0., x)
def relu_d(x): return (x > 0.)*1.

In [188]:
relu(np.array([3.,-3.])), relu_d(np.array([3.,-3.]))

(array([ 3.,  0.]), array([ 1.,  0.]))

In [189]:
def dist(a,b): return pow(a-b,2)
def dist_d(a,b): return 2*(a-b)

In [190]:
import pdb

In [191]:
eps = 1e-7
def x_entropy(pred, actual): 
    return -np.sum(actual * np.log(np.clip(pred, eps, 1-eps)))
def x_entropy_d(pred, actual): return -actual/pred

In [192]:
def softmax(x): return np.exp(x)/np.exp(x).sum()

In [193]:
def softmax_d(x):
    sm = softmax(x)
    res = np.expand_dims(-sm,-1)*sm
    res[np.diag_indices_from(res)] = sm*(1-sm)
    return res

In [194]:
test_preds = np.array([0.2,0.7,0.1])
test_actuals = np.array([0.,1.,0.])
nnet.categorical_crossentropy(test_preds, test_actuals).eval()

array(0.35667494393873245)

In [195]:
x_entropy(test_preds, test_actuals)

0.35667494393873245

In [196]:
test_inp = T.dvector()
test_out = nnet.categorical_crossentropy(test_inp, test_actuals)
test_grad = theano.function([test_inp], T.grad(test_out, test_inp))

In [197]:
test_grad(test_preds)

x_entropy_d(test_preds, test_actuals)

pre_pred = random(oh_x_rnn[0][0].shape)
preds = softmax(pre_pred)
actual = oh_x_rnn[0][0]

loss=x_entropy
loss_d=x_entropy_d

np.allclose(softmax_d(pre_pred).dot(loss_d(preds,actual)), preds-actual)

softmax(test_preds)

nnet.softmax(test_preds).eval()

test_out = T.flatten(nnet.softmax(test_inp))

test_grad = theano.function([test_inp], theano.gradient.jacobian(test_out, test_inp))

test_grad(test_preds)

softmax_d(test_preds)

act=relu
act_d = relu_d

loss=x_entropy
loss_d=x_entropy_d

In [198]:
def scan(fn, start, seq):
    res = []
    prev = start
    for s in seq:
        app = fn(prev, s)
        res.append(app)
        prev = app
    return res

In [199]:
scan(lambda prev,curr: prev+curr, 0, range(5))

[0, 1, 3, 6, 10]

### Set up training

In [200]:
inp = oh_x_rnn
outp = oh_y_rnn
n_input = vocab_size
n_output = vocab_size

In [201]:
inp.shape, outp.shape

((133406, 8, 203), (133406, 8, 203))

In [202]:
def one_char(prev, item):
    # Previous state
    tot_loss, pre_hidden, pre_pred, hidden, ypred = prev
    # Current inputs and output
    x, y = item
    pre_hidden = np.dot(x,w_x) + np.dot(hidden,w_h)
    hidden = act(pre_hidden)
    pre_pred = np.dot(hidden,w_y)
    ypred = softmax(pre_pred)
    return (
        # Keep track of loss so we can report it
        tot_loss+loss(ypred, y),
        # Used in backprop
        pre_hidden, pre_pred, 
        # Used in next iteration
        hidden, 
        # To provide predictions
        ypred)

In [203]:
def get_chars(n): return zip(inp[n], outp[n])
def one_fwd(n): return scan(one_char, (0,0,0,np.zeros(n_hidden),0), get_chars(n))

In [204]:
# "Columnify" a vector
def col(x): return x[:,newaxis]

def one_bkwd(args, n):
    global w_x,w_y,w_h

    i=inp[n]  # 8x86
    o=outp[n] # 8x86
    d_pre_hidden = np.zeros(n_hidden) # 256
    for p in reversed(range(len(i))):
        totloss, pre_hidden, pre_pred, hidden, ypred = args[p]
        x=i[p] # 86
        y=o[p] # 86
        d_pre_pred = softmax_d(pre_pred).dot(loss_d(ypred,y))  # 86
        d_pre_hidden = (np.dot(d_pre_hidden, w_h.T) 
                        + np.dot(d_pre_pred,w_y.T)) * act_d(pre_hidden) # 256

        # d(loss)/d(w_y) = d(loss)/d(pre_pred) * d(pre_pred)/d(w_y)
        w_y -= col(hidden) * d_pre_pred * alpha
        # d(loss)/d(w_h) = d(loss)/d(pre_hidden[p-1]) * d(pre_hidden[p-1])/d(w_h)
        if (p>0): w_h -= args[p-1][3].dot(d_pre_hidden) * alpha
        w_x -= col(x)*d_pre_hidden * alpha
    return d_pre_hidden

In [205]:
scale=math.sqrt(2./n_input)
w_x = normal(scale=scale, size=(n_input,n_hidden))
w_y = normal(scale=scale, size=(n_hidden, n_output))
w_h = np.eye(n_hidden, dtype=np.float32)

In [206]:
overallError=0
alpha=0.0001
for n in range(10000):
    res = one_fwd(n)
    overallError+=res[-1][0]
    deriv = one_bkwd(res, n)
    if(n % 1000 == 999):
        print ("Error:{:.4f}; Gradient:{:.5f}".format(
                overallError/1000, np.linalg.norm(deriv)))
        overallError=0

Error:42.5186; Gradient:2.07188
Error:40.9153; Gradient:3.03420
Error:37.3498; Gradient:3.70488
Error:35.5773; Gradient:2.91971
Error:34.2216; Gradient:3.24145
Error:33.5074; Gradient:2.98216
Error:32.6993; Gradient:4.16023
Error:32.7745; Gradient:4.05214
Error:32.9028; Gradient:2.90355
Error:32.1005; Gradient:2.90705


## Keras GRU

In [208]:
model=Sequential([
        GRU(n_hidden, return_sequences=True, input_shape=(cs, vocab_size),
                  activation='relu', inner_init='identity'),
        TimeDistributed(Dense(vocab_size, activation='softmax')),
    ])
model.compile(loss='categorical_crossentropy', optimizer=Adam())

In [209]:
model.fit(oh_x_rnn, oh_y_rnn, batch_size=64, nb_epoch=1)

Epoch 1/1
133406/133406 [==============================] - 22s - loss: 2.5788    


In [210]:
get_nexts_oh(' this is')

[' ', 't', 'h', 'i', 's', ' ', 'i', 's']


['t', 'h', 'e', 's', ' ', 'i', 'n', ' ']

## Theano GRU

In [211]:
W_h = id_and_bias(n_hidden)
W_x = init_wgts(n_input, n_hidden)
W_y = wgts_and_bias(n_hidden, n_output)
rW_h = init_wgts(n_hidden, n_hidden)
rW_x = wgts_and_bias(n_input, n_hidden)
uW_h = init_wgts(n_hidden, n_hidden)
uW_x = wgts_and_bias(n_input, n_hidden)
w_all = list(chain.from_iterable([W_h, W_y, uW_x, rW_x]))
w_all.extend([W_x, uW_h, rW_h])

In [212]:
def gate(x, h, W_h, W_x, b_x):
    return nnet.sigmoid(T.dot(x, W_x) + b_x + T.dot(h, W_h))

In [213]:
def step(x, h, W_h, b_h, W_y, b_y, uW_x, ub_x, rW_x, rb_x, W_x, uW_h, rW_h):
    reset = gate(x, h, rW_h, rW_x, rb_x)
    update = gate(x, h, uW_h, uW_x, ub_x)
    h_new = gate(x, h * reset, W_h, W_x, b_h)
    h = update*h + (1-update)*h_new
    y = nnet.softmax(T.dot(h, W_y) + b_y)
    return h, T.flatten(y, 1)

In [214]:
[v_h, v_y], _ = theano.scan(step, sequences=t_inp, 
                            outputs_info=[t_h0, None], non_sequences=w_all)

In [215]:
error = nnet.categorical_crossentropy(v_y, t_outp).sum()
g_all = T.grad(error, w_all)

In [216]:
upd = upd_dict(w_all, g_all, lr)
fn = theano.function(all_args, error, updates=upd, allow_input_downcast=True)

In [217]:
err=0.0; l_rate=0.1
for i in range(len(X)): 
    err+=fn(np.zeros(n_hidden), X[i], Y[i], l_rate)
    if i % 1000 == 999: 
        l_rate *= 0.95
        print ("Error:{:.2f}".format(err/1000))
        err=0.0

Error:28.15
Error:23.78
Error:21.15
Error:23.13
Error:22.09
Error:22.14
Error:21.46
Error:21.20
Error:22.24
Error:21.32
Error:20.93
Error:20.70
Error:20.92
Error:20.31
Error:22.89
Error:21.13
Error:20.69
Error:21.75
Error:19.52
Error:19.13
Error:20.27
Error:18.51
Error:22.33
Error:21.16
Error:17.59
Error:20.25
Error:19.27
Error:20.77
Error:20.46
Error:20.27
Error:19.46
Error:19.38
Error:18.94
Error:17.74
Error:19.40
Error:19.01
Error:19.60
Error:19.68
Error:19.24
Error:18.71
Error:19.29
Error:19.24
Error:19.78
Error:20.42
Error:21.25
Error:21.73
Error:18.09
Error:18.90
Error:20.58
Error:24.54
Error:20.15
Error:18.64
Error:20.53
Error:18.74
Error:19.22
Error:19.01
Error:19.40
Error:18.92
Error:19.18
Error:19.21
Error:19.66
Error:18.91
Error:20.97
Error:20.57
Error:20.35
Error:18.41
Error:19.49
Error:20.57
Error:20.80
Error:19.12
Error:18.86
Error:18.96
Error:20.84
Error:20.76
Error:20.02
Error:19.59
Error:20.18
Error:20.85
Error:19.77
Error:19.48
Error:20.33
Error:19.74
Error:18.91
Erro

# Combined Weights

In [218]:
W = (shared(np.concatenate([np.eye(n_hidden), normal(size=(n_input, n_hidden))])
            .astype(np.float32)), init_bias(n_hidden))

rW = wgts_and_bias(n_input+n_hidden, n_hidden)
uW = wgts_and_bias(n_input+n_hidden, n_hidden)
W_y = wgts_and_bias(n_hidden, n_output)
w_all = list(chain.from_iterable([W, W_y, uW, rW]))

In [219]:
def gate(m, W, b): return nnet.sigmoid(T.dot(m, W) + b)

In [220]:
def step(x, h, W, b, W_y, b_y, uW, ub, rW, rb):
    m = T.concatenate([h, x])
    reset = gate(m, rW, rb)
    update = gate(m, uW, ub)
    m = T.concatenate([h*reset, x])
    h_new = gate(m, W, b)
    h = update*h + (1-update)*h_new
    y = nnet.softmax(T.dot(h, W_y) + b_y)
    return h, T.flatten(y, 1)

In [221]:
[v_h, v_y], _ = theano.scan(step, sequences=t_inp, 
                            outputs_info=[t_h0, None], non_sequences=w_all)

In [222]:
def upd_dict(wgts, grads, lr): 
    return OrderedDict({w: w-g*lr for (w,g) in zip(wgts,grads)})

In [223]:
error = nnet.categorical_crossentropy(v_y, t_outp).sum()
g_all = T.grad(error, w_all)

In [224]:
upd = upd_dict(w_all, g_all, lr)
fn = theano.function(all_args, error, updates=upd, allow_input_downcast=True)

In [226]:
err=0.0; l_rate=0.001
for i in range(len(X)): 
    err+=fn(np.zeros(n_hidden), X[i], Y[i], l_rate)
    if i % 1000 == 999: 
        print ("Error:{:.2f}".format(err/1000))
        err=0.0

Error:18.62
Error:19.07
Error:19.82
Error:17.84
Error:17.68
Error:18.15
Error:17.63
Error:17.93
Error:18.99
Error:18.03
Error:18.02
Error:18.33
Error:18.18
Error:17.57
Error:21.23
Error:18.59
Error:19.59
Error:19.22
Error:17.37
Error:17.35
Error:18.27
Error:17.07
Error:20.19
Error:19.58
Error:16.57
Error:19.53
Error:18.42
Error:18.73
Error:18.32
Error:17.97
Error:17.77
Error:17.92
Error:17.40
Error:16.10
Error:17.87
Error:17.08
Error:17.88
Error:17.93
Error:17.64
Error:17.32
Error:17.62
Error:17.59
Error:17.54
Error:18.87
Error:19.30
Error:19.86
Error:16.47
Error:16.99
Error:19.12
Error:24.49
Error:18.20
Error:16.89
Error:18.72
Error:17.25
Error:17.55
Error:17.28
Error:17.74
Error:17.07
Error:17.83
Error:17.60
Error:17.96
Error:17.32
Error:17.62
Error:18.32
Error:18.21
Error:16.49
Error:17.36
Error:18.50
Error:19.20
Error:17.56
Error:17.40
Error:17.43
Error:19.25
Error:18.80
Error:18.55
Error:17.20
Error:16.81
Error:18.52
Error:17.32
Error:16.96
Error:17.63
Error:17.26
Error:16.48
Erro